In [ ]:
GCS_PATH="/home/apsisdev/ansary/DATASETS/APSIS/NID/segment/"
#----------------
# imports
#---------------
import tensorflow as tf
import random
import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import segmentation_models as sm

import numpy as np
import matplotlib.pyplot as plt
#from kaggle_datasets import KaggleDatasets
from glob import glob
from tqdm.auto import tqdm

%matplotlib inline

In [ ]:
#-------------
# GCS and files and synth
#-------------
def get_tfrecs(_path):
    gcs_pattern=os.path.join(_path,'*.tfrecord')
    file_paths = tf.io.gfile.glob(gcs_pattern)
    random.shuffle(file_paths)
    return file_paths

train_recs =[]
eval_recs=[]
# synth
#GCS_PATH = KaggleDatasets().get_gcs_path('aps-nid-segment-data')  

gcs_path=os.path.join(GCS_PATH,"tfrecords")
recs =get_tfrecs(gcs_path)
train_recs+=recs[20:]
eval_recs +=recs[:20]

print(len(eval_recs),len(train_recs))

In [ ]:
#-------------------------------------------------------------
# Detect hardware, return appropriate distribution strategy
#-------------------------------------------------------------

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    tf.config.optimizer.set_jit(True)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
#-------------------------
# train and data paras
#-----------------------
EPOCHS          = 500
if strategy.num_replicas_in_sync==1:
    BATCH_SIZE = 32
else:
    BATCH_SIZE = 16 * strategy.num_replicas_in_sync

STEPS_PER_EPOCH = (len(train_recs)*128)//BATCH_SIZE
EVAL_STEPS      = (len(eval_recs)*128)//BATCH_SIZE
print("Steps:",STEPS_PER_EPOCH)
print("Eval Steps:",EVAL_STEPS)

In [ ]:
#-------------
# data function with tf.data api
#-------------
DIM=(256,256,3)
def data_input_fn(recs): 
    '''
      This Function generates data from gcs
    '''
    
    def _parser(example):
        feature ={  'image'  : tf.io.FixedLenFeature([],tf.string),
                    'mask' : tf.io.FixedLenFeature([],tf.string),
                    'label' : tf.io.FixedLenFeature([2],tf.int64),
                    'bbox' : tf.io.FixedLenFeature([8],tf.int64)
        }     
        parsed_example=tf.io.parse_single_example(example,feature)
        # image
        image_raw=parsed_example['image']
        image=tf.image.decode_png(image_raw,channels=3)
        image=tf.cast(image,tf.float32)/255.0
        image=tf.reshape(image,DIM)
        
        # mask
        mask=parsed_example['mask']
        mask=tf.image.decode_png(mask,channels=1)
        mask=tf.cast(mask,tf.float32)/255.0
        mask=tf.reshape(mask,(DIM[0],DIM[1],1))
        
        # label
        label=parsed_example['label']
        label=tf.cast(label,tf.float32)

        
        outputs={"label":label,"sigmoid":mask}
        return image,outputs

    dataset = tf.data.TFRecordDataset(recs)
    dataset = dataset.map(_parser)
    dataset = dataset.shuffle(2048,reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

train_ds  =   data_input_fn(train_recs)
eval_ds   =   data_input_fn(eval_recs)

In [ ]:
#-----------------------
# visual
#-----------------------
import cv2

for x,y in eval_ds.take(1):
    l=y["label"]
    m=y["sigmoid"]
    data=np.squeeze(m[0])
    plt.imshow(data)
    plt.show()
    print(np.unique(data))
    # image
    data=np.squeeze(x[0])
    plt.imshow(data)
    plt.show()
    
    print(l[0])
    

# Modeling

```python
```

In [ ]:
def get_model(data_dim=(256,256,3),num_class=2):
    unet=sm.Unet("densenet121",input_shape=data_dim,encoder_weights=None,classes=1)
    inp  =unet.input
    # class
    label=unet.get_layer(name="relu").output
    label = tf.keras.layers.GlobalAveragePooling2D()(label)
    label=tf.keras.layers.Dense(num_class,activation="softmax",name="label")(label)
    # mask
    mask=unet.output
    model=tf.keras.Model(inputs=inp,outputs=[label,mask])
    return model

In [ ]:
model=get_model()
model.summary()

In [ ]:
#---------------
# callbacks
#---------------
# weight file path
weight_path="segment.h5"

# reduces learning rate on plateau
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1,
                                                  cooldown= 10,
                                                  patience=10,
                                                  verbose =1,
                                                  min_lr=0.1e-7)
# saves the model
model_autosave = tf.keras.callbacks.ModelCheckpoint(filepath=weight_path, 
                                                   monitor='val_loss', 
                                                   verbose=1, 
                                                   save_best_only=True, 
                                                   mode='min')

# early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(patience=15, 
                                                  verbose=1, 
                                                  mode = 'auto') 
callbacks= [model_autosave,lr_reducer,early_stopping]
                 
losses = {
"label":tf.keras.losses.CategoricalCrossentropy(),
"sigmoid": tf.keras.losses.BinaryCrossentropy(),
}
#---------------
# compile
#---------------    
with strategy.scope():
    # model
    model = get_model()
    # compile
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),loss=losses)

In [ ]:
history = model.fit(train_ds,
                    epochs=EPOCHS,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    verbose=1, 
                    validation_data=eval_ds,
                    validation_steps=EVAL_STEPS, 
                    callbacks=callbacks)

In [ ]:
# from IPython.display import FileLink
# FileLink(r'segment.h5')